In [1]:
import re
from urllib.parse import urlparse
import requests
from requests.exceptions import SSLError
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
r = requests.get('https://raw.githubusercontent.com/HD2i/biomedical-blockchain/master/README.md')
links = re.findall(r'\((.*?)\)', r.text)
links = links + [x.replace('<img src="', '').replace('" width="80"', '').replace('"/>', '').replace('>', '').replace(' width="200"', '') for x in re.findall(r'\[(.*?)\]', r.text)]

In [3]:
noise = [')', '"', '/', '*']
distinct_links = []
for link in links:
    if (link[-1:]) in noise:
        link = link[0:-1]
    if (link[-1:]) in noise:
        link = link[0:-1]
    distinct_links.append(link)

distinct_links = list(set(distinct_links))

In [4]:
def create_request(session, link, verify=True):
    request = requests.Request('GET', link, headers=headers)
    prepared_request = session.prepare_request(request)
    r = session.send(prepared_request, verify=verify)
    return r

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

for link in distinct_links:
    if link[0:4] != 'http': 
        continue
#     if link != 'https://enome.io': continue

    session = requests.Session()
    try:          
        r = create_request(session, link, verify=True)
        if r.status_code != 200:
            print('******************')
            print(link)
            print(r.status_code)
            print('******************')
    except SSLError as e:
        # catches wonky SSL certificates.  site works, just has trouble with security and requests.
        try:
            r = create_request(session, link, verify=False)
            if r.status_code != 200:
                print('******************')
                print(link)
                print(r.status_code)
                print('******************')
        except Exception as e:
            print(e)
    except Exception as e:
        print('******************')
        print(link)
        print(r)
        print(e)
        print('******************')

******************
http://dci.mit.edu/assets/papers/eckblaw.pdf
404
******************
